### 机器学习
###### 一、定义与流程
（一）机器学习的定义 （二）机器学习的流程 （三）机器学习的集成库

（一）机器学习的定义     
计算器从提供的数据集中发现 x和y直接的关系，从而使得它预测出来的y越来越准确的过程

（二）机器学习的流程    
1、明确任务目的（有监督学习：分类问题、回归问题 ； 无监督学习：聚类问题 另外还有半监督学习和强化学习，暂时未涉及）     
2、数据预处理（清洗：处理缺失值，异常值，错误值；转化：归一化、标准化）     
sklearn.preprocessing / pandas的缺失值处理能力 fillna dropna findna   
3、特征工程（特征即维度，可增加可减少可线性变化，比如采用主成分分析（PCA）和线性判别分析（LDA），其也算数据预处理的一部分）   
sklearn.decomposition  
sklearn.feature_selection      
4、依据目的和数据集特征选出算法（对应关系可见下图）       
sklearn.neighbors   
sklearn.neural_network    
skleran.naive_bayes   
skleran.cluster   
skleran.linear_model        
5、模型训练（将数据分为训练集和测试集，一般测试集占据20-30%）     
sklearn.model_selection   
6、评估优化   
sklearn.metrics    
7、实际上线          
8、接收反馈再优化          

In [ ]:
from IPython.display import Image, display
image_path = r"D:\1-script\3-PYTHON\image存储jupyter在用的图片存放于此\机器学习的流程.png"
display(Image(filename = image_path,width=500, height=500))

（三）机器学习的集成库

In [ ]:
# !pip install scikit-learn

### 机器学习
###### 二、数据预处理
（一）归一化 （二）标准化 （三）正则化 （四）简易推荐使用哪种数据预处理方式

（一）归一化     
1.归一化定义  2.归一化方法说明与适用场景

1. 归一化定义   
归一化是一种数据预处理技术，用于调整数值数据的范围，以便将其标准化到一个特定的范围，通常是 0 到 1。这种处理可以使数据在不同的尺度下具有可比性，同时有助于机器学习算法更好地理解和处理数据。

2. 归一化方法说明与适用场景

In [ ]:
# 假设存在一个特征数组为x_list , 其中包含N个x元素
x_list = [50,75,100,200]
print("原始数值:", x_list)

# 方法一 最小-最大归一化（Min-Max Scaling）:
# 目的：将特征缩放到 [0, 1] 范围内，对于异常值太敏感，因为他依赖于最大最小值做计算 ，这种情况推荐用标准化
# 适用情况：当数据不遵循正态分布时；当特征的最大值和最小值是已知的。
x_preprocess = [(x-min(x_list)) /  (max(x_list)-min(x_list)) for x in x_list]
print(f'最小-最大归一化预处理后的数值: {x_preprocess}')

# 方法二 对数归一化:
# 目的：缩放数据，使得长尾数据更加紧密呈现近似正态分布，但值不会固定在[0,1]
# 适用情况：适合处理长尾数据 然后sklearn中并无直接的工具 可以numpy对数化之后再最大最小值处理
# 对于满足对数分布或指数分布的数据，对数归一化可以使其接近正态分布。但是，假如原始数据中存在负数或零，则不能直接使用对数函数（因为 log(x) 要求 x大于0），需要先将所有数值都转换为正数之后再归一化。
# 对数正态分布是典型的长尾
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 模拟一些具有长尾分布的数据
data = np.random.exponential(scale=2, size=1000).reshape(-1, 1)

# 应用对数变换
data_log_transformed = np.log1p(data)  # np.log1p(x) 计算 log(1 + x)，用于处理包含0的数据

# 使用 MinMaxScaler 进行归一化
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_log_transformed)

# 绘制原始数据和处理后数据的直方图 一个会拖出一个长长的尾巴 而非骤然归零 对数后的更加紧密 趋向于正态分布
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(data, bins=50, color='blue', alpha=0.7)
plt.title("Original Data")

plt.subplot(1, 2, 2)
plt.hist(data_normalized, bins=50, color='green', alpha=0.7)
plt.title("Log Transformed and Normalized Data")

plt.show()

# 方法三 反正切归一化
# 目的：利用反正切函数（即 arctan）来将数据映射到固定的范围内，反正切的值在− π/2 到 π/2 ； 为了让归一化后的数据落在更常用的区间 [0, 1] 内，可以进一步将 arctan 的输出进行变换。
# 适用情况：适合处理数据范围广泛或包含离群值的数据集 重尾分布的数据时非常有用，如金融数据或具有异常值的环境数据
# sklearn 一样没有直接能用的， 但是可以基于库的基础类，自定义一个转化器
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class ArctanScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # 无需学习任何参数
        return self
    
    def transform(self, X, y=None):
        # 应用反正切归一化
        return (np.arctan(X) + np.pi / 2) / np.pi

# 使用示例
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_regression

# 生成随机数据
X, y = make_regression(n_samples=100, n_features=1, noise=0.1)

# 创建归一化管道
pipeline = Pipeline([
    ('arctan_scaler', ArctanScaler())
])

# 归一化数据
X_scaled = pipeline.fit_transform(X)

print("Original Data Sample:", X[:5])
print("Scaled Data Sample:", X_scaled[:5])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

# 生成对数正态分布数据
data_lognorm = np.random.lognormal(mean=0, sigma=1, size=1000)

# 生成指数分布数据
data_exp = np.random.exponential(scale=1, size=1000)

# 绘制直方图
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data_lognorm, bins=30, alpha=0.7, color='blue')
plt.title("Log-Normal Distribution")
plt.subplot(1, 2, 2)
plt.hist(data_exp, bins=30, alpha=0.7, color='green')
plt.title("Exponential Distribution")
plt.show()

# 绘制Q-Q图
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
stats.probplot(np.log(data_lognorm), dist="norm", plot=plt)
plt.title("Q-Q Plot of Log of Log-Normal Data")
plt.subplot(1, 2, 2)
stats.probplot(data_exp, dist="expon", plot=plt)
plt.title("Q-Q Plot of Exponential Data")
plt.show()

（二）标准化 

1. 标准化定义    
调整数据的特征使之具有零均值和单位方差，形成“标准正态”分布的特性，使其均值为 0，方差为 1。

In [ ]:
# 方法一 Z得分归一化（Standardization）:
# 目的：将数据的均值转换为 0，标准差转换为 1。
# 适用情况：数据遵循正态分布；在不知道数据的最小值和最大值的情况下，也即对于异常值不敏感
x_list = [50,75,100,200]
import numpy as np
x_array = np.array(x_list)
x_preprocess = [(x - np.mean(x_array))/np.std(x_array) for x in x_list]
print(f'Z得分归一化预处理后的数值: {x_preprocess}')

from sklearn.preprocessing import StandardScaler
import numpy as np

# 示例数据
X = np.array([[1.0, -1.0, 2.0],
              [2.0, 0.0, 0.0],
              [0.0, 1.0, -1.0]])

# 创建标准化转换器对象
scaler = StandardScaler()

# 训练标准化参数并转换数据
X_scaled = scaler.fit_transform(X)

# 显示标准化后的数据
print("Standardized Data:\n", X_scaled)

# 检查均值和标准差
print("Mean of each feature after scaling:", X_scaled.mean(axis=0))
print("Std deviation of each feature after scaling:", X_scaled.std(axis=0))

（三）正则化   

1. 正则化定义     
在数据预处理中，L1和L2常常指的是L1范数和L2范数的应用，特别是在特征缩放（scaling）和归一化（normalization）的过程中。这些方法用于调整数据特征的比例，以提高算法的性能和稳定性。

In [ ]:
# 方法一 单位向量归一化（L2 Normalization）:
# 目的：将特征向量的长度缩放至 1。
# 适用情况：在处理文本数据或需要计算向量空间中的相似度时。
x_list = [50,75,100,200]
x_array = np.array(x_list)
l2_norm = np.linalg.norm(x_array)
x_preprocess = x_array / l2_norm
print("单位向量归一化预处理后的数值:", x_preprocess)

import numpy as np
from sklearn.preprocessing import Normalizer

# 创建一个示例数据集
data = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# 初始化 Normalizer 对象
normalizer = Normalizer(norm='l2')

# 使用 fit_transform 方法归一化数据
normalized_data = normalizer.fit_transform(data)

print("原始数据:")
print(data)
print("归一化后的数据:")
print(normalized_data)

In [ ]:
# 附录展示正态分布的检验方式
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

# 生成一个正态分布样本数据
data = np.random.normal(loc=0, scale=1, size=1000)

# 绘制直方图
plt.hist(data, bins=30, alpha=0.75, color='blue')
plt.title('Histogram')
plt.show()

# 绘制Q-Q图
stats.probplot(data, dist="norm", plot=plt)
plt.title('Q-Q Plot')
plt.show()

# 进行Shapiro-Wilk检验
shapiro_test = stats.shapiro(data)
print(f"Shapiro-Wilk Test: Statistic={shapiro_test[0]:.4f}, p-value={shapiro_test[1]:.4f}") # 输出中的统计量越接近1，p值越大，越不拒绝数据来自正态分布的原假设。

# 进行Kolmogorov-Smirnov检验（假设为正态）
ks_test = stats.kstest(data, 'norm', args=(np.mean(data), np.std(data)))
print(f"Kolmogorov-Smirnov Test: Statistic={ks_test[0]:.4f}, p-value={ks_test[1]:.4f}") # 此检验比较实际数据与正态分布的累积分布函数。p值较大时，我们接受数据符合正态分布的假设。

（四）简易推荐使用哪种数据预处理方式

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

def analyze_dataset(dataset):
    # 检查数据类型
    if not isinstance(dataset, pd.Series) and not isinstance(dataset, np.ndarray):
        return "Dataset should be a pandas Series or numpy array."
    
    # 强制转换为 numpy array 进行处理
    data = np.array(dataset).flatten()
    
    # 检测空值
    if np.any(np.isnan(data)):
        return "Data contains NaN values. Please handle them before analysis."

    # 基本描述统计
    descriptive_stats = {
        'mean': np.mean(data),
        'std': np.std(data),
        'min': np.min(data),
        'max': np.max(data),
        'median': np.median(data),
        'skewness': stats.skew(data),
        'kurtosis': stats.kurtosis(data)
    }
    
    # 检测正态分布
    k2, p_norm = stats.normaltest(data)
    is_normal = p_norm > 0.05
    
    # 检测指数分布
    _, p_exp = stats.kstest(data, 'expon', args=(np.min(data), np.mean(data)))
    is_exponential = p_exp > 0.05
    
    # 检测均匀分布
    _, p_uniform = stats.kstest(data, 'uniform', args=(np.min(data), np.max(data)-np.min(data)))
    is_uniform = p_uniform > 0.05
    
    # 检测异常值
    q75, q25 = np.percentile(data, [75 ,25])
    iqr = q75 - q25
    lower_bound = q25 - 1.5 * iqr
    upper_bound = q75 + 1.5 * iqr
    outliers = np.where((data < lower_bound) | (data > upper_bound))
    has_outliers = len(outliers[0]) > 0
    
    # 推荐预处理方法
    if has_outliers:
        preprocess_recommendation = "Consider using robust scaling due to outliers."
    elif is_normal:
        preprocess_recommendation = "Data seems normal. Standard scaling is recommended."
    elif is_exponential or descriptive_stats['std'] > descriptive_stats['mean']:
        preprocess_recommendation = "Data seems exponential or skewed. Log transformation and MinMax scaling are recommended."
    else:
        preprocess_recommendation = "MinMax scaling can be used."
    
    # 业务领域推荐
    if is_normal:
        business_context = "Likely applications: Any domain requiring normality assumptions (e.g., many statistical tests, Gaussian processes)."
    elif is_exponential:
        business_context = "Likely applications: Quality control, survival analysis, risk assessment."
    elif is_uniform:
        business_context = "Likely applications: Simulations where uniform distribution is expected."
    elif has_outliers:
        business_context = "Likely applications: Financial transactions, social media metrics, real estate prices."
    else:
        business_context = "General data processing."
    
    return {
        'Descriptive Statistics': descriptive_stats,
        'Is Normal': is_normal,
        'Is Exponential': is_exponential,
        'Is Uniform': is_uniform,
        'Has Outliers': has_outliers,
        'Preprocess Recommendation': preprocess_recommendation,
        'Business Context': business_context
    }

# 使用示例
data = pd.Series(np.random.lognormal(mean=0, sigma=1, size=1000))
analysis_results = analyze_dataset(data)
analysis_results

### 机器学习
###### 三、特征选择
（一）卡方校验（二）t检验 （五）主成分分析（PCA） （六）线性判别分析（LDA）

（零）何为特征选择                

复数的特征 每个特征是一个x，理论上会有无数的xi 每个xi都有一个对应的yj 目标值        
我们需要讨论的问题是 x 和 y两个变量之间是否存在强关联 越关联的特征才应该被选择为模型的特征 不然容易过拟合    
所以我们需要某种计算方式指导我们

卡方检验：分类任务 XY离散型     
适用场景：   
独立性检验：检验两个分类变量之间是否存在关联。   
拟合优度检验：检验观测数据与理论分布之间的拟合程度。    

t 检验：回归任务 XY连续型 检验参数是否显著                 
使用场景：
单样本 t 检验：检验样本均值与已知总体均值的差异。          
双样本 t 检验：检验两个独立样本均值之间的差异。           
配对样本 t 检验：检验成对样本（如实验前后测量）均值之间的差异。             
线性回归中的 t 检验：检验回归系数是否显著。            

（一）卡方校验

1. 卡方检验原理

卡方检验基于独立性假设，独立性假设假定当两个变量不存在相关性的时候 他们同时发生的概率= x单独发生的概率 * y单独发生的概率    
公式表现为：P(X=Xi and Y=Yj)=P(X=Xi)×P(Y=Yj)   
公式解释  ：P 代表概率 简单理解为 Xi出现的概率= Xi出现的样本数/样本总数N ； N则代表样本总数            

2. 观察频数（Observed Frequency）

任意一个分类的数据表 可以通过列联表的方式 做出一个x值为i时 y值为j出现的频次 这是由样本提供的真实的频度值 被我们直接观察到 故而命名观察频度

3. 期望频数（Expected Frequency）根据独立性假设计算得到

基于独立性假设可以通过公式的变换算出xiyj的不相关的时候的期望频度值              
**步骤一** 认知P(X=Xi and Y=Yj)= Eij/N          
Eij代表 xiyj时出现的期望频次 / 联合总样本数 即可算出 P（X=Xi and Y=Yj）          
因此公式推导如下             
P(X=Xi and Y=Yj)= Eij/N =P(X=Xi)×P(Y=Yj)             
Eij = P(X=Xi)×P(Y=Yj) * N           
 
**步骤二**          
同理P(X=Xi) = Xi出现的频次/X的总样本数  =N样本数          
   P(Y=Yj) = Yj出现的频次/Y的总样本数  =N样本数        
   
如下图的观察频度所示 就是行列总计    
Xi出现的频次 = Ri   
Yj出现的频次 = Ci    
因此公式推导如下   
Eij = Ri/N×Cj/N * N    

**步骤三**      
最终按照数学知识，简化算式     
Eij = Ri/N×Cj/N * N = Ri*Cj/N    

In [ ]:
from IPython.display import Image, display
image_path = r"D:\1-script\3-PYTHON\image存储jupyter在用的图片存放于此\列联表（观察频度）.jpg"
display(Image(filename = image_path,width=100, height=100))

4. 卡方值的测算

通过实际样本拿到观察频数             
通过独立性假设获取期望频数              
如果两者的差异越大 则代表着现实于假设差距甚远 代表着两种并不是假设的无关 

接下来就是量化两者的差异 计算公式如下：          
公式   ：    
希腊字母 chiχ² 即卡方值/卡方统计量 =∑ (0i - Ei)² / Ei       
公式解释：       
∑代表求和 即所有xy的观察频次和期望频次的差异的平方总和        
²是为了确保 无论是大于差异还是小于差异都被算作差异 而非互相抵消     
/Ei是为了保证 不同频数的贡献是相对的 使得整体卡方值具有一致的尺度 

5. 卡方分布 与 显著性

通过上述假设 和公式推导 我们将变量x和y之间是否存在显著相关性的问题转化为了一个 卡方值 ，现在需要解决的是卡方值处于多少才能算是显著    
为了理解这一点 再次引入概率论的卡方分布知识

卡方分布是一种常见的概率分布，广泛应用于假设检验中 同时正态分布的平方 = 卡方分布。 为了确定卡方值是否显著，我们需要以下几个步骤：       
（1）计算卡方统计量      
（2）确定自由度          
（3）选择显著性水平（𝛼）：通常选择 0.05（95% 置信水平），也可以选择 0.01 或 0.10 等其他水平       
（4）查找临界值：在卡方分布表中查找对应自由度和显著性水平的临界值（critical value）。       
（5）比较卡方统计量与临界值：如果卡方统计量大于临界值，则拒绝零假设，认为观察频数和期望频数之间的差异是显著的。         

因为自由度的提升，同样的显著性水平要求的临界值会更高 ，反之当自由度过小 往往发生在较小的样本时 会导致卡方值更容易超过临界值 呈现假阳性
故而进一步引入   
Yates修正（Yates' correction）    
考虑使用其他非参数检验方法，如Fisher确切检验（Fisher's exact test），它在小样本情况下更为合适。   

6. PYTHON 语句使用 

In [ ]:
# 6.1 使用统计库 理解计算单个x和y的关系
import numpy as np
from scipy.stats import chi2_contingency

# 创建列联表
data = np.array([[10, 20], [30, 40]])

# 执行卡方检验
chi2, p, dof, expected = chi2_contingency(data)

# 设定显著性水平
alpha = 0.05

# 输出结果
print(f"Chi-Square Statistic: {chi2}")
print(f"P-Value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)

# 判断结果 当 x不取某个值时，观察到y值的概率小于设定的显著性水平  就代表 不取x为某值 则y低于p的概率出现值 这代表可以推翻原始假设
if p <= alpha:
    print("拒绝原假设，结果具有统计显著性。")
else:
    print("不拒绝原假设，结果不具有统计显著性。")

In [ ]:
import pandas as pd
odf = pd.crosstab(data.data[:,0],data.target)
dof = (odf.shape[0] - 1) * (odf.shape[1] - 1)
row = odf.sum(axis = 1)
column = odf.sum(axis = 0)
n = odf.sum().sum()

import numpy as np
edf = np.zeros(odf.shape)
for i in range(0,odf.shape[0]):
    for j in range(0,odf.shape[1]):
        edf[i,j] = row.iloc[i]*column.iloc[j]/n
        
chi = ((odf-edf)**2/edf).sum().sum()
chi

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import KBinsDiscretizer
from scipy.stats import chi2_contingency
from sklearn.feature_selection import SelectKBest,chi2

# 加载数据集
data = load_iris()
X, y = data.data, data.target

# 假设特征是非负的（卡方校验要求）
X = np.abs(X)

# 使用 KBinsDiscretizer 对特征进行离散化处理
binning = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
X_binned = binning.fit_transform(X)


# 计算列联表和卡方统计量
chi2_values,p,dof,expected = chi2_contingency(X_binned, y)
# 打印结果
print("手动计算的卡方统计量:", chi2_values)
print("手动计算的P值:", p_values)



手动计算的卡方统计量: 711.1902865100636
手动计算的P值: 2.253773920299271e-14


In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import KBinsDiscretizer

# 加载数据集
data = load_iris()
X, y = data.data, data.target

# 假设特征是非负的（卡方校验要求）
X = np.abs(X)

# 使用 KBinsDiscretizer 对特征进行离散化处理
binning = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
X_binned = binning.fit_transform(X)

# 检查离散化后的数据分布
print("离散化后的特征:\n", X_binned)

# 使用 SelectKBest 和 chi2 进行特征选择
selector = SelectKBest(score_func=chi2, k=2)
X_new = selector.fit_transform(X_binned, y)

print("原始特征形状:", X.shape)
print("选择后的特征形状:", X_new.shape)
print("选择的特征索引:", selector.get_support(indices=True))
print("卡方统计量:", selector.scores_)
print("P值:", selector.pvalues_)

离散化后的特征:
 [[2. 6. 0. 0.]
 [1. 4. 0. 0.]
 [1. 5. 0. 0.]
 [0. 4. 0. 0.]
 [1. 6. 0. 0.]
 [3. 7. 1. 1.]
 [0. 5. 0. 0.]
 [1. 5. 0. 0.]
 [0. 3. 0. 0.]
 [1. 4. 0. 0.]
 [3. 7. 0. 0.]
 [1. 5. 1. 0.]
 [1. 4. 0. 0.]
 [0. 4. 0. 0.]
 [4. 8. 0. 0.]
 [3. 9. 0. 1.]
 [3. 7. 0. 1.]
 [2. 6. 0. 0.]
 [3. 7. 1. 0.]
 [2. 7. 0. 0.]
 [3. 5. 1. 0.]
 [2. 7. 0. 1.]
 [0. 6. 0. 0.]
 [2. 5. 1. 1.]
 [1. 5. 1. 0.]
 [1. 4. 1. 0.]
 [1. 5. 1. 1.]
 [2. 6. 0. 0.]
 [2. 5. 0. 0.]
 [1. 5. 1. 0.]
 [1. 4. 1. 0.]
 [3. 5. 0. 1.]
 [2. 8. 0. 0.]
 [3. 9. 0. 0.]
 [1. 4. 0. 0.]
 [1. 5. 0. 0.]
 [3. 6. 0. 0.]
 [1. 6. 0. 0.]
 [0. 4. 0. 0.]
 [2. 5. 0. 0.]
 [1. 6. 0. 0.]
 [0. 1. 0. 0.]
 [0. 5. 0. 0.]
 [1. 6. 1. 2.]
 [2. 7. 1. 1.]
 [1. 4. 0. 0.]
 [2. 7. 1. 0.]
 [0. 5. 0. 0.]
 [2. 7. 0. 0.]
 [1. 5. 0. 0.]
 [7. 5. 6. 5.]
 [5. 5. 5. 5.]
 [7. 4. 6. 5.]
 [3. 1. 5. 5.]
 [6. 3. 6. 5.]
 [3. 3. 5. 5.]
 [5. 5. 6. 6.]
 [1. 1. 3. 3.]
 [6. 3. 6. 5.]
 [2. 2. 4. 5.]
 [1. 0. 4. 3.]
 [4. 4. 5. 5.]
 [4. 0. 5. 3.]
 [5. 3. 6. 5.]
 [3. 3. 4. 5.]
 [6. 4. 5. 5.]


### 机器学习
###### 四、算法原理解释、适用问题与SKlearn写法

（一）KNN 近邻算法

1. 定义与适用情况   
（1） knn 近邻算法 通过各种数学公式找到数据点直接的距离 再根据k个相近点 投票找到未知点可能的分类   
（2） 适合数据小 维度少 数据分布相对平均 不会局部稀疏的 数据集 最为基础的算法 用于回归也用于分类   
（3） 判断点与点之间的距离存在复数的方式 包括但不限于欧式距离，切比雪夫距离，曼哈顿距离，以及三合一的闵可夫斯基距离

2. 手动展示knn 以一个分类问题为例

In [ ]:
# 0. 判断任务目的 通过已知数据分类未知宝可梦的类别 所以是分类问题 

# 1. 拿到数据源并简单观察
# 1.1 引入数据源 
import pandas as pd
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )
df_unknown = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '神秘宝可梦' )

In [ ]:
# 1.2 观察数据源
# 在只使用对战数据的情况下 体重和身高的缺失值无需处理 反之则需要处理
df_known.info() # 684 non-null 

In [ ]:
# 2.演示KNN模型逻辑 现在假设我们只需要没有未知值的对战数据 无缺失且特征都需要 直接开始跑模型
# 选取对战的特征 作为 x值 分类的宝可梦为y值 选取3条数据出来 
r1 = df_known.loc[1,'对阵虫系':'对阵水系']   
r2 = df_known.loc[100,'对阵虫系':'对阵水系'] 
r3 = df_known.loc[3,'对阵虫系':'对阵水系'] 

# 计算2点距离 等于计算 所有特征的距离 平方相加 开根号
sum( [(r1[i]-r2[i])**2 for i in range(0,len(r1))] ) 
import math
print('r1和r2的距离是：', math.sqrt(sum( [(r1[i]-r2[i])**2 for i in range(0,len(r1))] ) ) ) # 3.09
print('r1和r3的距离是：', math.sqrt(sum( [(r1[i]-r3[i])**2 for i in range(0,len(r1))] ) ) ) # 3.44


In [ ]:
# 3. 写成函数 对整个数据集应用 判断与未知的距离
import math
def knn(row):
    known = row['对阵虫系':'对阵水系']
    unknown = df_unknown.loc[0,'对阵虫系':'对阵水系'] # 假设这个是记载未知r1值的x值
    return  math.sqrt(sum( [(known[i]-unknown[i])**2 for i in range(0,len(known))] ) ) 

df_known['knn'] = df_known.apply(knn,axis = 1) # 一行一个y 复数x 对于 每个x 即行的每个列做操作 所以axis = 1 强制理解

# 获取应用数据集的index 找到与他最近的k位邻居 k = 10
df_known.iloc[df_known['knn'].sort_values().head(7).index, :].groupby('主分类').agg(主分类计数=('主分类', 'count'))
                                                                                     # 新列名      使用列名  函数操作
# k=10 10个草系 4个虫系 所以结果是草系
# k=7  3个草系  4个虫系 所以结果是虫系

df_known.iloc[df_known['knn'].sort_values().head(7).index, :]['主分类'].mode()[0] # 取众数知道什么系列最多

3. 采用SKlearn库来使用KNN算法

In [ ]:
# 1. 明确目的为分类
# 2. 数据预处理
# 2.1 引入数据源 
import pandas as pd
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )
df_unknown = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '神秘宝可梦' )
# 3 特征工程

In [ ]:
# 4 选择k近邻算法 
# 5 模型训练
# 5.1 拆分训练集与测试集
from sklearn.model_selection import train_test_split
dfk_train,dfk_test = train_test_split(df_known,         # 已知数据集
                                     test_size = 0.3,   # 测试集比例 一般0.2-0.3
                                     random_state = 42, # 不想每次抽取都随机 任意整数固定数据集划分
                                     shuffle = True,   # 默认True 代表随机打乱数据集 反之固定顺序抽取没有随机性 random_state 参数失去意义
                                     #stratify = df_known['主分类'] #  控制训练集和测试集中各类的比例与完整数据集中的比例相同 
                                                                   # 不支持固定顺序抽取 即shuffle = False
                                     ) 

# train_test_split??

In [ ]:
# 5.2 生成knn算法对象
from sklearn.neighbors import KNeighborsClassifier 
k = 7
knn = KNeighborsClassifier(n_neighbors = k) # 后续直接更改k值不需要再次生成knn，也不需要重新knn.fit，他存放的是距离值，k只决定几个邻居投票

# 5.3 获取需要x和y值
x = dfk_train.loc[:,'对阵虫系':'对阵水系']
y = dfk_train['主分类']

# 5.4 进行拟合 生成模型
k_model = knn.fit(x,y)

In [ ]:
# 6 评估优化
# k_model是已经获取的模型 用测试集拿去测试下准确率

# 6.1 测试集预测
y_predicted = k_model.predict(dfk_test.loc[:,'对阵虫系':'对阵水系'])

# 6.2 评估模型 即与真实分类做比较
y_real = dfk_test['主分类'].values
print(f'k值为{k}时，预测准确率是：',round(
                                    (y_predicted == y_real).sum() # 统计True的值为178
                                    /y_real.size ,                # 全部测试集长度为211
                                    4) 
     )

# 6.3 或者直接使用sk封装的评估工具
from sklearn.metrics import accuracy_score
accuracy_score( dfk_test['主分类'], y_predicted) # 先写y_true，再写y_predicted

# print('模型的精确度是：',round(Accuracy,4))

4. 多种距离算法参数运用   
默认metric 为闵可夫斯基距离 调整p值 使得 1为曼哈顿 2为欧式 无穷大为切比雪夫距离   
其他距离算法 请调整metric参数，从默认的前往其他算法    
    "euclidean"  欧氏距离              sqrt(sum((x-y)^2))

    "manhattan"  曼哈顿距离         sum(|x-y|)

    "chebyshev"  切比雪夫距离       max(|x-y|)

    "minkowski"  闵可夫斯基距离      sum(w*|x-y|^p)^(1/p)

    "seuclidean"  标准欧氏距离          sqrt(sum((x-y)^2/V))

    "mahalanobis"  马哈拉诺比斯距离    sqrt((x-y)'V^-1(x-y))

    "hamming"    海明（汉明）距离       sum(|w*(x-y)|^p)^(1/p)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
knn = KNeighborsClassifier(n_neighbors = 2,p=1)

# 生成一个用于距离演算的df
df1 =  pd.DataFrame( data = [[5,5,5],[5,1,3],[2,2,2]],columns = ['射门','速度','评分'])

# 构建模型 装填训练集数据 包含的 x y 值
model = knn.fit(df1.loc[0:1 , '射门':'速度'] , df1.loc[0:1 , '评分'] )

# 调用模型的kneighbors方法 获取单个点的距离 返回距离值和 测试集的索引
distances, index = model.kneighbors(df1.loc[[2] , '射门':'速度']) # 返回几个和 n_neighbors 参数有关 只返回你要求的数量 而非全部计算过的
print(f'和\n{df1.iloc[index[0]]}\n距离分别是{distances[0]}')

# 验证出p=1的时候为曼哈顿距离
p = 1 
x1 = np.array(df1.loc[0:1 , '射门':'速度'])
x2 = np.array(df1.loc[[2] , '射门':'速度'])

for _ in x1-x2:
    # 手动计算下曼哈顿距离 每个维度相减取绝对值最后相加
    print(sum([abs(x) for x in _ ]))

# metric 参数设置其他距离名字，默认 闵可夫斯基距离
# weights 参数设置每个邻居计算的权重 典型的是求取反比倒数 weights = distance 默认weights = uniform 

### 机器学习
###### 三、算法原理解释、适用问题与SKlearn写法

（二）朴素贝叶斯算法

1. 定义与适用情况   
（1） 朴素贝叶斯算法 适合高纬稀疏数据 高纬指维度特征很多 稀疏指大部分的特征值都是空或者0  
（2） 朴素在假设每个维度之间的关系是相互独立的 但是现实中不可能存在两个特征之间毫无关系  
（3） 贝叶斯算法基于贝叶斯定理 下面详细解释

2. 贝叶斯定理  
（1）描述两个条件概率之间的关系  即当一件事情（条件）发生时另外一件事情（条件）发生的概率 先发生的事情位于|的后面    
（2）常见的公式会是  
P(A|B) = P(B|A)* P(A) /P(B)   
P(A|B)表示在B发生的条件下A发生的概率  即需要推算的后验概率    
P(B|A)标识在A发生的条件下B发生的概率 即条件概率 似然 通常是 关心的分类中出现选定的特征的样本概率  
P(A)代表着A事件不受任何因素影响的情况下发生的概率 可以简易理解为 关心的分类/样本总数     
P(B)代表着边缘似然 也可以说是B事件不受因素影响发生的概率 朴素贝叶斯中通常视其为常数 无需计算     

以垃圾邮件举例 邮件由若干的文本组成 每一个文本即为一个特征 假设现在仅取发票和折扣两个词作为特征 用于识别一封邮件是否为广告   
就是在判断 邮件文本中包含发票与折扣的时候（前一个事件发生了），是广告这个事情发生的概率    
套用贝叶斯公式写成   
P(是广告|文本中包含发票与折扣) 正比于 P(文本包含发票与折扣|是广告)*P(是广告) / P(文本中包含发票与折扣)   
此项公式可以进一步细分维度 即 假设 两个特征不存在相关性 从而推算 是广告 文本包含发票与折扣 这一个事情发生的概率约等于 是广告 文本中包含发票概率 * 是广告 文本中包含折扣概率   
**** 仅仅模型是朴素的  贝叶斯公式可不是

3. 朴素贝叶斯模型的优缺点  
优点：  
（1）高纬稀疏数据也可以处理 因为简化维度之间的相关性   
（2）面对大数据集也能做到高效率 因为计算量非常的小 仅仅计算 先验概率（你关注的分类在样本中的出现概率）和条件概率（特征/关注分类）  就可以推算出后验概率    
（3）对缺失数据不会很敏感 即使某个维度未出现 也可以通过拉普拉斯平滑的方式加常数规避0值    
缺点：   
（1）过度简化维度相关性 导致性能的显著下降 尤其在面对维度本身相关性很强的数据集时   

4. 使用朴素贝叶斯之前 必须要完成特征工程 即自然语言处理（NLP） 我们在这里介绍两种处理方式     
（1）词袋模式：忽视词与词的前后顺序 仅统计词出现的频度      
（2）N-GRAM模式：关注词与词出现的先后顺序 N代表统计的词连续的数量      

5. 手动展示 英文 中文的分词处理 与他向量化的过程

In [ ]:
# 5.1 假设存在如下英文语句需要做处理

import pandas as pd
data=[
     ["I love natural language processing","正面" ],
     ["Language processing is fascinating","正面" ],
     ["I dislike artificial intelligence","负面"  ],
     ["Machine learning is hard",         "负面"  ],       
     ]
df = pd.DataFrame(data,columns = ('language','status'))

# 5.2 调用sklearn中的工具 不用的话 可以使用中文分词工具 结巴 在频度统计列表也可以 ;英文直接可以使用 字符串对象的split方法
from sklearn.feature_extraction.text import CountVectorizer # 统计 向量化器

# 依赖于预设对象生成实例 countvector
countvector = CountVectorizer(
                        stop_words = None,  # 默认统计所有分词
                        ngram_range = (1,1), # 默认生成词袋模型 range如果等于(1,2)就代表1个单词 或者连续的两个单词都要用于生成词汇表
                        token_pattern = r"(?u)\b\w+\b" # 默认使用的r'(?u)\b\w\w+\b' 会导致一个字母的不被留下来
                        # lowercase = False  # 默认True 即全部小写化
                        # max_df            # 大于这个频度的词不统计 浮点数代表词汇在文档中的最大比例 整数代表词汇在文档中的最大出现次数
                        # min_df            # 小于这个频度的词不统计
                        # max_features      # 指定最大的特征数量 按照频道排序取最大的特征
                        # vocabulary        # 指定仅统计这个里面的单词
                        # binary            # 二进制向量 所有频度都归1
                        # dtype             # 指定输出格式
                        )

# 调用countvector的 fit方法 去解析一个词汇表 全部分类的词都拿去解析
countvector.fit(df['language'])
vocabulary = countvector.get_feature_names_out() # 解析后才能调用这个方法获取vocabulary

# 使用transform转化向量矩阵 调用toarray()方法实现肉眼观察
x = countvector.transform(df['language'])
x.toarray() 

# 或者直接使用fit_transform方法一次实现 赋值CountVectorizer的vocabulary 并输出向量矩阵
x = countvector.fit_transform(df['language'])
print('词汇列表如下：\n',countvector.get_feature_names_out())
x.toarray()



6. 基于5向量化的东西做一个预测

In [ ]:
# 6.1 计算先验概率
# 保留下向量结果 用于后续的条件概率/似然计算
df['向量结果'] = list(x.toarray())

# 增加类别聚合 计算对应的先验和似然的前置数据
dfagg = df.groupby('status').agg(类别样本数 = ('status','count') ,词汇频度 = ('向量结果','sum')     ).reset_index()
dfagg['总样本数'] = df.shape[0]
dfagg['先验概率'] = dfagg['类别样本数']/dfagg['总样本数']
dfagg

In [ ]:
# 6.2 计算似然

单个特征的似然我们步骤式理解下
1. 为何我们要计算似然  
P(C∣D)=P(D∣C)⋅P(C) / P(D)   
简化为：P(C∣x)∝P(C)⋅P(x1∣C)⋅P(x 2∣C)* …… * P(x n∣C)    
2. 带入下案例看看我们在算什么   
案例中 C 即为正面或者负面       
案例中 共4个样本 求取的似然是 是正面/负面 分类的条件发生时 每个特征/词汇向量 出现的概率   
3. 单个特征似然的公式（w=c 都代表特征）    
对于一个词 𝑤 在类别 𝐶下的条件概率 𝑃(𝑤|𝐶) 其计算公式应该是：   
P(w∣C)= 类别 C 中词 w 的计数+α / 类别 C 中所有词的计数总和+α×词汇表大小）   
**这里的分母 不是类别C的样本数 2 而是整个分类C下所有词的计数总和 他们才是计算词频的分母**    
4. 带入下案例计算出love    
我们先求取类别：正面中的 love出现的概率    
类别 正面 中词 love 的计数：1   
类别 正面 中所有词的计数总和：9   
词汇表大小：13    
概率正常应该是：1/9  = 11.11%    
拉普拉斯平滑 我们通常设定 α= 1 +1平滑也可以有+2平滑     
拉普拉斯平滑概率：1+1/9+1*13 = 9.09%    
5. 附录案例的 词汇表 和 词汇向量化的矩阵  
词汇列表：   
['artificial' 'dislike' 'fascinating' 'hard' 'i' 'intelligence' 'is'
 'language' 'learning' 'love' 'machine' 'natural' 'processing']    
词汇向量化的矩阵：    
array([    
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1], # 正面   
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1], # 正面   
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], # 负面   
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0], # 负面    
      ],dtype=int64)    
未处理的样本数据：    
data=[        
     ["I love natural language processing","正面" ],      
     ["Language processing is fascinating","正面" ],        
     ["I dislike artificial intelligence","负面"  ],       
     ["Machine learning is hard",         "负面"  ],         
     ]

In [ ]:
# 6.2.1 写成脚本计算出所有词汇包含的特征的条件概率 又称似然
# 前面已经完成了似然所需的向量化矩阵 现在开始计算剩余的部分

# 类别 正面 中所有词的计数总和
import numpy as np
dfagg['类别下词汇总计数'] = dfagg['词汇频度'].apply(lambda x : np.sum(x))

# 计算词汇表的大小 
vocab_len = len(vocabulary) # 13

# 增加拉普拉斯平滑参数 Alpha
Alpha = 1

# 最后来计算每个词的概率
dfagg['条件概率'] = (dfagg['词汇频度']+1)/(dfagg['类别下词汇总计数']+Alpha*vocab_len)

# 检查数据
dfagg

In [ ]:
import numpy as np
vocabulary =  ['artificial', 'dislike', 'fascinating' ,'hard' ,'i' ,'intelligence', 'is',
 'language', 'learning', 'love', 'machine', 'natural', 'processing']

# 新样本
new_sample = "I love machine learning"

# 将新样本转换为词频向量
def text_to_vector(text, vocabulary):
    vector = np.zeros(len(vocabulary),dtype = 'int64')
    words =  [text.lower() for text in text.split()]
    for word in words:
        if word in vocabulary:
            vector[vocabulary.index(word)] += 1
    return vector

new_sample_vector = text_to_vector(new_sample, vocabulary)

In [ ]:
def calculate_posterior(dfagg, new_sample_vector):
    posteriors = []
    for _, row in dfagg.iterrows():
        prior = row['先验概率']
        likelihood = np.prod([
            row['条件概率'][i] ** new_sample_vector[i]
            for i in range(len(new_sample_vector))
        ])
        posterior = prior * likelihood
        posteriors.append(posterior)
    return posteriors

posteriors = calculate_posterior(dfagg,new_sample_vector)
posteriors/sum(posteriors) # array([[0.29333794, 0.70666206]])

7. 使用sklearn库算出概率

In [ ]:
# 忘记库下模块中的类别 可以用以下方式
import sklearn.naive_bayes
import inspect
inspect.getfile(sklearn.naive_bayes)

dir(sklearn.naive_bayes)
help(sklearn.naive_bayes)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np
mnb = MultinomialNB( alpha = 1.0, # 指定拉普拉斯平滑参数值
                     fit_prior =True,  # 是否学习提供的类的先验概率 默认True 如果改为False 先验概率将全部相等
                     # class_prior = [0.2,0.8], # 指定后 不再受到数据的影响 以列表形式如[0.2,0.8]提供 类顺序自然排序 所以需要检查
                     # force_alpha = False   # 强制force alpha 为浮点数 默认default=False
                   )

# 进行特征工程 这个和上面保持一致
import pandas as pd
data=[
     ["I love natural language processing","正面" ],
     ["Language processing is fascinating","正面" ],
     ["I dislike artificial intelligence","负面"  ],
     ["Machine learning is hard",         "负面"  ],       
     ]
df = pd.DataFrame(data,columns = ('language','status'))

# 调用sklearn中的工具 不用的话 可以使用中文分词工具 结巴 在频度统计列表也可以 ;英文直接可以使用 字符串对象的split方法
from sklearn.feature_extraction.text import CountVectorizer # 统计 向量化器

# 依赖于预设对象生成实例 countvector
countvector = CountVectorizer(
                        stop_words = None,  # 默认统计所有分词
                        ngram_range = (1,1), # 默认生成词袋模型 range如果等于(1,2)就代表1个单词 或者连续的两个单词都要用于生成词汇表 容易导致模型太复杂 一般最多就(1,2)
                        token_pattern = r"(?u)\b\w+\b" # 默认使用的r'(?u)\b\w\w+\b' 会导致一个字母的不被留下来
                        # lowercase = False  # 默认True 即全部小写化
                        # max_df            # 大于这个频度的词不统计 浮点数代表词汇在文档中的最大比例 整数代表词汇在文档中的最大出现次数 
                        # min_df            # 小于这个频度的词不统计 一般舍弃低频词和过高频词
                        # max_features      # 指定最大的特征数量 按照频道排序取最大的特征
                        # vocabulary        # 指定仅统计这个里面的单词
                        # binary            # 二进制向量 所有频度都归1
                        # dtype             # 指定输出格式
                        )

# 调用countvector的 fit方法 去解析一个词汇表 全部分类的词都拿去解析
countvector.fit(df['language'])
vocabulary = countvector.get_feature_names_out() # 解析后才能调用这个方法获取vocabulary

# 使用transform转化向量矩阵 调用toarray()方法实现肉眼观察
x = countvector.transform(df['language'])
x.toarray() 

# 或者直接使用fit_transform方法一次实现 赋值CountVectorizer的vocabulary 并输出向量矩阵
x = countvector.fit_transform(df['language'])
# print('词汇列表如下：\n',countvector.get_feature_names_out())
x.toarray()
y = df['status']

# 训练数据
model = mnb.fit(x,y)
print('训练数据中的类别：',model.classes_)
print('训练数据中的类别的先验概率：',np.exp(model.class_log_prior_)) # 值是log(0.5) 上面求取过的分类的0.5概率的对数 代表先验概率算法一致
# print(model.class_prior) # 生成的时候没指定所以是None
print('训练数据中的类别下每个特征对应的条件概率：\n',model.feature_log_prob_) # 故意保留对数 后面会解释为什么最好都用对数

# 将训练数据封装为一个df
dfmodel = pd.DataFrame({
    '类别':model.classes_ , 
    '先验概率':np.exp(model.class_log_prior_), 
    '特征的条件概率':list(np.exp(model.feature_log_prob_)), 
             })
# 如果调用时发现过长显示不完全 用这个
pd.set_option('display.max_colwidth', None)

# 调下上面手工代码计算出来的值  发现两者完全一致
# print(dfagg['条件概率']）
print('对应的手工自写代码的结果：')
print([np.log(i) for i in dfagg['条件概率']])
print('两者完全一致')

8. 补充说明 为什么必须使用对数 而非原始概率值      
（1）避免浮点数下溢，概率相乘导致很小很小的数 最后溢出近0值             
（2）对数变化后的公式 表现从原始的乘法       
P(x1,x2,...,xn∣y)=P(x1∣y)⋅P(x2|y)⋅...⋅P(xn∣y) 改为        
logP(x1,x2,...,xn∣y)=logP(x1∣y)+logP(x2∣y)+...+logP(xn∣y) 加法比乘法提高数值稳定性              
（3）在优化和损失函数计算过程中，使用对数值可以简化很多计算。         
（4） 更易于理解和解释 对数概率的范围是负无穷大到0，而不是0到1           

9. 实际预测下

In [ ]:
new_sample = ["I love machine learning ye"]
x = countvector.transform(new_sample)
x.toarray()

model.predict_proba(x)
model.predict(x)

10. 邮件分类实战

In [ ]:
# 导入数据集
import pandas as pd
df1 = pd.read_excel(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\trec06cn-1.xlsx")
df2 = pd.read_excel(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\trec06cn-2.xlsx")


In [ ]:
# 合并清理缺失值
df = pd.concat([df1,df2],axis = 0)
df.dropna(subset = ['邮件正文'],how = 'any',inplace = True)
df.groupby('分类').count()

# 预处理分词
import jieba 
df['分词'] = df['邮件正文'].apply(lambda x : ' '.join(jieba.cut(x)) )


In [ ]:
# 导入三方库
from sklearn.naive_bayes import MultinomialNB # 模型
from sklearn.feature_extraction.text import CountVectorizer # 词汇特征处理
from sklearn.pipeline import Pipeline # 管道
from sklearn.model_selection import train_test_split # 训练测试分离
from sklearn.model_selection import GridSearchCV # 网格搜索
from sklearn.preprocessing import LabelEncoder   # 特征预处理
from sklearn.feature_selection import SelectKBest, chi2 # 特征选择

# 预处理y值
label = LabelEncoder()
df['分类数值化'] = label.fit_transform(df['分类'])

# 建立管道 将多个步骤合为一体 统一搜索
pipe = Pipeline([
    ('vector', CountVectorizer()),
    ('select', SelectKBest(chi2)), # 添加特征选择步骤 此处采用卡方校验
    ('nb', MultinomialNB())
])

# 提供参数 以步骤名称__参数名称 区分参数从属于哪个步骤
param_grid = {  'vector__ngram_range':((1,1),(1,2))   ,   # 一般最多(1,2) 否则容易过拟合 模型过于复杂 但有时候确实需要(1,3)
                'vector__max_df': (0.75, 1.0),                  # max_df 用于过滤高频词
                'vector__min_df': (1, 2),                       # min_df 用于过滤低频词
                'select__k': (1000, 2000, 3000),                # 卡方校验下选择的特征数量
                'nb__alpha': (0.5, 1, 2),                       # 拉普拉斯平滑 即为贝叶斯模型中的正则惩罚项 调整来避免欠过拟合        
                'nb__fit_prior':(True,False),                  #  如果样本本身太偏离现实的概率 建议赋值 而非学习样本中的先验概率
             }

# 训练 测试分离
x_train,x_test,y_train,y_test = train_test_split(df['分词'],df['分类数值化'],test_size =0.2,random_state = 42)

# 生成网格搜索器
gscv = GridSearchCV(pipe,param_grid,cv=10,scoring = 'precision')

# 训练模型
gscv.fit(x_train,y_train)

In [ ]:
# 调用最好的模型 再次预测
best_nb = gscv.best_estimator_
y_predicit = best_nb.predict(x_test)

# 查看最好模型的参数组合
print('最好的模型参数',gscv.best_params_)
print('最好的分数',gscv.best_score_)

In [ ]:
from sklearn import metrics
print('F1分数',metrics.f1_score(y_test,y_predicit)         )
print('准确率',metrics.accuracy_score(y_test,y_predicit)   )
print('召回率',metrics.recall_score(y_test,y_predicit)     )
print('精确率',metrics.precision_score(y_test,y_predicit)  )
print(metrics.classification_report(y_test,y_predicit)     )
# average  = 'macro' 'micro' 'weighted' 

# 补充完整报告的解读方式

In [ ]:
print("选择的特征索引:", selector.get_support(indices=True))

In [ ]:
gscv.best_score_

In [ ]:
df.groupby(['分类']).count()

In [ ]:
from fbox_wt.user

### 机器学习
###### 四、快速调参手段
（一）KFload 交叉验证（二）网格搜索

（一）KFload 交叉验证   
1.交叉验证的作用  2.采用SKlearn库来实现随机的交叉验证 基本模板 3.k近邻为例展示调参数方法

1.交叉验证的作用     
交叉验证主要是为了评估 模型在不同的子集上的预测能力和稳定性，同时选择最优的模型参数 ，以及更为现实的数据量不足，否则也可以用train_test_split将一个大型的数据集直接分为训练，验证，测试    
一般日常工作中我们会选择10折，即n_solits = 10    

2.采用SKlearn库来实现随机的交叉验证 基本模板

In [ ]:
# 引入k折类 生成对象
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10,shuffle = True, random_state = 42)

# 读取任意数据集
import pandas as pd
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )

# 数据集重组
df = df_known.iloc[:20,0:7]
kf.split(df) # 返回的是 训练集 验证集 的 2个索引数组
for train_index,validate_index in kf.split(df_known.iloc[:20]):
    # print(type(validate_index))
    df_train = df.iloc[train_index,:]
    df_validate  = df.iloc[validate_index,:]
    print(f'本轮的训练集是 \n {df_train} \n本轮的测试集是 \n {df_validate}')


3. k近邻为例展示调参数方法

In [ ]:
# 读取任意数据集
import pandas as pd
import numpy as np
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )    
    

# 先完成测试集的分离
from sklearn.model_selection import train_test_split
df_raw_train,df_test = train_test_split(df_known,test_size = 0.2, shuffle = True, random_state = 42)

# 以k近邻为例 使用交叉验证主要是加强验证取n_neighbors = ? 最优 避免正好出现 n_neighbors = k 时 仅仅对于某种数据集分组最准确的情况
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 循环k值 在不同的拆分时的结果
k_ac_score = {}
for k in range(1,11,2):
    knn = KNeighborsClassifier(n_neighbors = k)
    
    # 训练集10折拆分
    from sklearn.model_selection import KFold
    kf = KFold(n_splits = 10,shuffle = True, random_state = 42)
    
    # 开始10折的循环
    ac_score_list = []
    for train_index,validate_index in kf.split(df_raw_train):
        # print('开始本轮的交叉验证')
        df_train    = df_raw_train.iloc[train_index,:]
        df_validate = df_raw_train.iloc[validate_index,:]
        # print(f'本轮的训练集是 \n {df_train} \n本轮的测试集是 \n {df_validate}')
        # 训练模型
        model = knn.fit(df_train.loc[:,'对阵虫系':'对阵水系'] , df_train['主分类'])
        ac_score = round(accuracy_score(df_validate['主分类'], model.predict(df_validate.loc[:,'对阵虫系':'对阵水系']) ),2)
        ac_score_list.append(ac_score)

    ac_score = round(np.mean(np.array(ac_score_list))    ,2)
    print(f'当k值为{k}时，准确率为{ac_score},每个折的准确率分别为{ac_score_list}')
    k_ac_score[k] = (ac_score,ac_score_list)

# 全部的k值都做过10折运算后 获取到了字典 在用max内置函数 key参数对于传递的值做解析后再比较 返回max的键为值的能力找到最合适的k值
max_key = max(k_ac_score, key=lambda k: k_ac_score[k][0])
print(f'最合适的k值为{max_key}')

（二）网格搜索

1. 网格搜索的定义     
基于交叉验证的复合技术，用于系统地遍历多种算法的参数组合，从而找到最佳的参数设置        
网格搜索的本质可以认为是上面以knn算法为例的超参数调优的模块化脚本 既然验证都是要知道在什么算法下 做哪些参数 采取几折的验证方式 自然可以定义为一个对象 提供参数 自动的去做 验证，省掉了重复写循环的麻烦      

2. sklearn中的GridSearchCV

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# 生成网格搜索对象
knn = KNeighborsClassifier()
gscv = GridSearchCV(knn,{'n_neighbors':range(1,10,2)},cv = 10)


# 读取任意数据集
import pandas as pd
import numpy as np
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )    
    

# 先完成测试集的分离
from sklearn.model_selection import train_test_split
df_raw_train,df_test = train_test_split(df_known,test_size = 0.2, shuffle = True, random_state = 42)
gscv.fit(df_raw_train.loc[:,'对阵虫系':'对阵水系'],df_raw_train['主分类'])

pd.DataFrame(gscv.cv_results_)

In [ ]:
gscv.best_params_

### 机器学习
###### 五、评估模型的方式
（一）准确率 accuracy_score （二）召回率指标 Recall 又称灵敏度 (Sensitivity)     
（三）精确率指标 Precision  （四）F1分数 F1Score
（五）roc曲线 （六）ruc曲线

（一）准确性指标 Accuracy_score

1. 定义与使用情况   
Accuracy= TP+TN/TP+TN+FP+FN  任意分类正确预测的数/样本总数量    
评估方法适用于二分类和多分类问题，并且在各类样本均衡的情况下特别有效。    
对于不平衡的分类问题，比如一个分类占比90%的数据集，那么我判断所有的数据都是某一类其准确性完全可能高于模型的预测结果    
所以还要引入精确率（Precision）、召回率（Recall）、F1 分数等综合评估   

2. 采用SKlearn库来使用accuracy指标

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score??

（二）召回率指标 Recall 又称灵敏度 (Sensitivity)

1. 定义与使用情况   
Recall = TP/TP+FN 正例预测正确的数量/实际正例的数量   
不平衡样本在现实世界中较多 准确率导致严重的误导 故而进一步引入 召回率 典型的比如分类罪犯与好人 分类病人和非病人 都是你不关注的负例（好人，非病人）占比样本中的绝大多数，导致了不平衡的发生，但是错误放过那些人的问题很大 所以会有召回率，也即对于识别出的正例的占比   
召回率侧重于最大限度地识别出所有正类样本。

2. 采用SKlearn库来使用Recall指标

In [ ]:
from sklearn.metrics import recall_score
recall_score??
# average  = 'macro' 'micro' 'weighted' 
# ……还有很多不同的平均值算法，这是准确率没有的参数 但是这个必须提供，否则无法解决多分类情况下的指标平均值计算

（三）精确率指标 Precision

1. 定义与使用情况    
Precision = TP/TP+FP 识别过程中真正例 占比 模型预测的正例     
不断强化模型准确的识别出实际正例 也可能导致不是正例的被分类为正例 典型的不应如此的场景为邮件分类，重要邮件被分类至垃圾导致错过      
精确率指标 侧重于最大限度减少误报     

2. 采用SKlearn库来使用Precision指标

In [ ]:
from sklearn.metrics import precision_score
precision_score??
# average  = 'macro' 'micro' 'weighted' 
# ……还有很多不同的平均值算法，这是准确率没有的参数 但是这个必须提供，否则无法解决多分类情况下的指标平均值计算

（四）F1分数 F1Score

1. 定义与使用情况    
F1Score = 2×（Precision×Recall） / （Precision+Recall）   
召回率和精确率是一对此消彼长的指标，而F1分数在精确率和召回率之间提供了一个平衡    
特别适合于那些对精确率和召回率同等重视的场景    

上述算法为调和平均值，利用了期强调较小数值的重要性时这一个特性，任何一个指标过小，都将导致整体值变小 唯有两者都搞才能使F1分数变高

2. 采用SKlearn库来使用F1Score指标

In [ ]:
from sklearn.metrics import f1_score
f1_score??
# average  = 'macro' 'micro' 'weighted' 
# ……还有很多不同的平均值算法，这是准确率没有的参数 但是这个必须提供，否则无法解决多分类情况下的指标平均值计算

### 机器学习  
###### 附录 课后习题   

###### 第5卷 机器学习 第五十三回   
1.请使用循环测试当K的值为 1~19 时，不同的K值预测的主分类并观察结果。

In [ ]:
# 1. 获取数据
import pandas as pd
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )
df_unknown = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '神秘宝可梦' )
# 2. 分一个训练测试用于额外的习题
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df_known,test_size = 0.2)

In [ ]:
# 3.定义一个knn算法
import math
def knn(row,x_unknown):
    x_known = row['对阵虫系':'对阵水系']
    return math.sqrt(sum([(x_unknown[i]-x_known[i])**2 for i in range(0,x_unknown.size)]) )   

In [ ]:
# 4. apply于每一行 
df_known['距离'] = df_known.apply(knn,axis =1,x_unknown = df_unknown.loc[0,'对阵虫系':'对阵水系'].squeeze())

In [ ]:
# 5.排序取k值 算出众数值 获得分类
df_known.sort_values(by = '距离',axis = 0,ascending = True).head(5)['主分类'].mode()

# 6.循环完成习题1
for k in range(1,20):
    classifier = df_known.sort_values(by = '距离',axis = 0,ascending = True).head(k)['主分类'].mode()[0]
    # print(f'当k值为{k}，这个宝可梦的分类为{classifier}')
    
# 7.问题升级，预测多个未知的k值 在不同的k值时的值 作为列放进去测试集
classifier_list = []
for index,row in df_test.iterrows():
    df_train['距离'] = df_train.apply(knn,axis =1,x_unknown = row['对阵虫系':'对阵水系'])
    classifier = [df_train.sort_values(by = '距离',axis = 0 ,ascending = True).head(k)['主分类'].mode()[0]  for k in range(1,4)]
    classifier_list.append(classifier)

df_test['分类'] = classifier_list

2.读取火焰纹章数据，预测未知的人物的职业

In [ ]:
# 1. 获取数据
import pandas as pd
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\fire_emblem_1.xlsx",
                         sheet_name = '已知角色' )
df_unknown = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\fire_emblem_1.xlsx",
                         sheet_name = '未知角色' )

# 2. 分一个训练测试用于额外的习题
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df_known,test_size = 0.3,#stratify = df_known['职业']
                                   )

# 3. 调取knn算法
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier()

x = df_train.loc[:,'力量':'行动点']
y = df_train.loc[:,'职业']

# 4.生成模型
model = knn.fit(x,y)
for k in range(1,8,2):
    # 下面两句语句造成的k值改变是等价的，同时改变knn和model的，且他们为同一类对象
    knn.n_neighbors = k 
    model.n_neighbors = k 
    
    # 5.预测
    y_predicted = model.predict(df_test.loc[:,'力量':'行动点'])
    
    # 6. 核验准确性
    from sklearn.metrics import accuracy_score
    ac_score = accuracy_score(df_test['职业'],y_predicted)
    print(f'当k值为{k}，精确度为{round(ac_score,3)*100}%')

    # 7. 实际预测 
    y_predicted = model.predict(df_unknown.loc[:,'力量':'行动点'])
    print(y_predicted)

In [ ]:
df_train

In [ ]:
# 1. 获取数据
import pandas as pd
df_known = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '已知宝可梦' )
df_unknown = pd.read_excel(io = r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\pokemon_knn.xlsx",
                         sheet_name = '神秘宝可梦' )
# 2. 分一个训练测试用于额外的习题
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df_known,test_size = 0.2)

# 3.定义一个knn算法
import math
def knn(row,x_unknown):
    x_known = row['对阵虫系':'对阵水系']
    return math.sqrt(sum([(x_unknown[i]-x_known[i])**2 for i in range(0,x_unknown.size)]) )   

# 4. apply于每一行 
df_known['距离'] = df_known.apply(knn,axis =1,x_unknown = df_unknown.loc[0,'对阵虫系':'对阵水系'].squeeze())

# 5.排序取k值 算出众数值 获得分类
df_known.sort_values(by = '距离',axis = 0,ascending = True).head(5)['主分类'].mode()

# 6.循环完成习题1
for k in range(1,20):
    classifier = df_known.sort_values(by = '距离',axis = 0,ascending = True).head(k)['主分类'].mode()[0]
    # print(f'当k值为{k}，这个宝可梦的分类为{classifier}')
    
# 7.问题升级，预测多个未知的k值 在不同的k值时的值 作为列放进去测试集
classifier_list = []
for index,row in df_test.iterrows():
    df_train['距离'] = df_train.apply(knn,axis =1,x_unknown = row['对阵虫系':'对阵水系'])
    classifier = [df_train.sort_values(by = '距离',axis = 0 ,ascending = True).head(k)['主分类'].mode()[0]  for k in range(1,4)]
    classifier_list.append(classifier)

df_test['分类'] = classifier_list

In [ ]:
df_train['距离'] = df_train.apply(knn,axis =1,x_unknown = row[0,'对阵虫系':'对阵水系'])